In [27]:
import pandas as pd

In [28]:
enduti_usuarios_anual_2020=pd.read_csv(r'data\conjunto_de_datos_endutih_2020_csv\conjuntos_de_datos\tr_endutih_usuario_anual_2020.csv', dtype="object", encoding='latin1')

In [29]:
enduti_usuarios_anual_2021=pd.read_csv(r'data\conjunto_de_datos_endutih_2021_csv\conjuntos_de_datos\tr_endutih_usuario_anual_2021.csv', dtype="object", encoding='latin1')

In [30]:
enduti_usuarios_anual_2022=pd.read_csv(r'data\conjunto_de_datos_endutih_2022_csv\conjunto_de_datos\tr_endutih_usuarios_anual_2022.csv', dtype="object", encoding='latin1')

In [31]:
enduti_usuarios_anual_2023=pd.read_csv(r'data\conjunto_de_datos_endutih_2023_csv\conjunto_de_datos\tr_endutih_usuarios_anual_2023.csv', dtype="object", encoding='latin1')

In [32]:
import pandas as pd

# Define the age bins and labels
age_bins = [6, 17, 24, 30, 35, 45, 55, 65, float('inf')]
age_labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

# List of dataframes and the years corresponding to them
dataframes = [
    ('2020', enduti_usuarios_anual_2020),
    ('2021', enduti_usuarios_anual_2021),
    ('2022', enduti_usuarios_anual_2022),
    ('2023', enduti_usuarios_anual_2023)
]

# Dictionary to store transformed dataframes
transformed_dataframes = {}

for year, df in dataframes:
    # Convert 'EDAD' to numeric
    df['EDAD'] = pd.to_numeric(df['EDAD'])
    
    # Create age groups
    df['GRUPO_EDAD'] = pd.cut(df['EDAD'], bins=age_bins, labels=age_labels)
    
    # Convert 'FAC_PER' to numeric
    df['FAC_PER'] = pd.to_numeric(df['FAC_PER'])
    
    # Check if the column name for the year is 'P7_19' or 'P7_21'
    p7_column = 'P7_21' if year == '2023' else 'P7_19'
    
    # Group by relevant columns including the P7 column
    grouped_df = df.groupby(['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO', p7_column], observed=False).agg({'FAC_PER': 'sum'}).reset_index()
    
    # Pivot the dataframe to have the P7 column values as separate columns
    pivoted_df = grouped_df.pivot_table(
        index=['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO'], 
        columns=p7_column, 
        values='FAC_PER', 
        fill_value=0,
        observed=False
    ).reset_index()
    
    # Rename columns for clarity
    pivoted_df.columns = ['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO', 'FAC_PER_P7_19_1', 'FAC_PER_P7_19_2']
    
    # Store the transformed dataframe in the dictionary
    transformed_dataframes[year] = pivoted_df

    # Display the shape of the transformed dataframe for each year
    print(f"Transformed dataframe for {year}: shape {pivoted_df.shape}")


Transformed dataframe for 2020: shape (2048, 6)
Transformed dataframe for 2021: shape (2048, 6)
Transformed dataframe for 2022: shape (2048, 6)
Transformed dataframe for 2023: shape (2048, 6)


In [33]:
# Assume transformed_dataframes is the dictionary with the transformed DataFrames
df_2020 = transformed_dataframes['2020']
df_2021 = transformed_dataframes['2021']
df_2022 = transformed_dataframes['2022']
df_2023 = transformed_dataframes['2023']

# Ensure that the categories are aligned (e.g., categorical columns are of the same type)
categorical_features = ['GRUPO_EDAD', 'SEXO', 'ESTRATO']

# Convert 'ENT' to integer type
for df in [df_2020, df_2021, df_2022, df_2023]:
    df['ENT'] = df['ENT'].astype(int)

# Convert categorical columns to string for consistency
for col in categorical_features:
    for df in [df_2020, df_2021, df_2022, df_2023]:
        df[col] = df[col].astype(str)


In [34]:
# Merge the combined "prev" dataset with the 2022 target
merged_df_2020_2022 = pd.merge(
    df_2020, 
    df_2022[['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO', 'FAC_PER_P7_19_1', 'FAC_PER_P7_19_2']],
    on=['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO'],
    how='inner'
)

# Merge the combined "prev" dataset with the 2023 target
merged_df_2021_2023 = pd.merge(
    df_2021, 
    df_2023[['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO', 'FAC_PER_P7_19_1', 'FAC_PER_P7_19_2']],
    on=['GRUPO_EDAD', 'SEXO', 'ENT', 'ESTRATO'],
    how='inner'
)


In [35]:
from catboost import CatBoostRegressor, Pool
# Concatenate the two merged datasets
combined_df = pd.concat([merged_df_2020_2022, merged_df_2021_2023], ignore_index=True)
combined_df = combined_df[combined_df['FAC_PER_P7_19_1_y'] > 0]

# Split features and target
X_train = combined_df.drop(columns=['FAC_PER_P7_19_1_y', 'FAC_PER_P7_19_2_y'])
y_train = combined_df['FAC_PER_P7_19_1_y']


# Specify categorical columns for CatBoost
cat_features_indices = [X_train.columns.get_loc(col) for col in categorical_features]

# Create a CatBoost Pool for FAC_PER_P7_19_1
train_pool_1 = Pool(data=X_train, label=y_train, cat_features=cat_features_indices)

# Initialize the CatBoost Regressor for FAC_PER_P7_19_1
model_1 = CatBoostRegressor(iterations=2000, learning_rate=0.1, depth=16, verbose=0)

# Train the model for FAC_PER_P7_19_1
model_1.fit(train_pool_1)

# Make predictions for FAC_PER_P7_19_1
predictions_1 = model_1.predict(train_pool_1)

# Display the first few predictions alongside the actual values for FAC_PER_P7_19_1
print("Predictions for FAC_PER_P7_19_1:")
for pred, actual in zip(predictions_1[:20], y_train[:20]):
    print(f"Predicted: {pred:.2f}, Actual: {actual}")

KeyboardInterrupt: 

In [26]:
from sklearn.metrics import r2_score
import numpy as np

# Function to calculate mean percentage error (MPE) with zero-value handling
def mean_percentage_absolute_error(y_true, y_pred):
    # Mask to exclude zero values in y_true to prevent division by zero
    mask = y_true != 0
    return np.mean(abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# Calculate R-squared and mean percentage error for FAC_PER_P7_19_1
r2_1 = r2_score(y_train, predictions_1)
mpe_1 = mean_percentage_absolute_error(y_train, predictions_1)

print("\nEvaluation Metrics for FAC_PER_P7_19_1:")
print(f"R-squared: {r2_1:.4f}")
print(f"Mean Percentage Error (MPE): {mpe_1:.2f}%")


Evaluation Metrics for FAC_PER_P7_19_1:
R-squared: 0.9550
Mean Percentage Error (MPE): 29.92%
